<a href="https://colab.research.google.com/github/zakariarafi/Data_Science_Practice/blob/master/Multiclass_Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
# Membaca data csv dan mengubahnya menjadi dataframe
df = pd.read_csv('imdb_indonesian.csv')
# Menghapus kolom 'judul_film'
df = df.drop(columns=['judul_film'])
df.head ()

,ringkasan_sinopsis,genre
0,Raden Mas Said putra sulung Tumenggung Wilarik...,Drama
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,Drama
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,Drama
3,POL menceritakan kisah hidup yang luar biasa d...,Drama
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,Drama


In [7]:
category = pd.get_dummies(df.genre)
df_baru = pd.concat([df, category], axis=1)
df_baru = df_baru.drop(columns='genre')
df_baru

,ringkasan_sinopsis,Drama,Horor,Komedi,Laga,Romantis
0,Raden Mas Said putra sulung Tumenggung Wilarik...,1,0,0,0,0
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,1,0,0,0,0
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,1,0,0,0,0
3,POL menceritakan kisah hidup yang luar biasa d...,1,0,0,0,0
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,1,0,0,0,0
...,...,...,...,...,...,...
1000,Winter in Tokyo berpusat pada kehidupan Ishida...,0,0,0,0,1
1001,Markonah melarikan diri ke Jakarta karena akan...,0,0,0,0,1
1002,"Tempat aking lebih dari 36 jam, Last Night ada...",0,0,0,0,1
1003,Proyek baru ini adalah tentang seorang lelaki ...,0,0,0,0,1


In [8]:
# Mengubah nilai-nilai dataframe menjadi numpy array
sinopsis = df_baru['ringkasan_sinopsis'].values
label = df_baru[['Drama', 'Horor', 'Komedi', 'Laga', 'Romantis']].values

In [9]:
from sklearn.model_selection import train_test_split
# Membagi data menjadi set latihan dan set uji
sinopsis_latih, sinopsis_test, label_latih, label_test = train_test_split(sinopsis, label, test_size=0.2)

In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# Menginisialisasi tokenizer dengan jumlah kata maksimal 5000 dan kata out of vocabulary ditandai dengan 'x'
tokenizer = Tokenizer(num_words=5000, oov_token='x')

In [11]:
# Melakukan tokenisasi pada data latihan dan uji
tokenizer.fit_on_texts(sinopsis_latih)
tokenizer.fit_on_texts(sinopsis_test)

In [12]:
# Mengubah teks menjadi sequence
sekuens_latih = tokenizer.texts_to_sequences(sinopsis_latih)
sekuens_test = tokenizer.texts_to_sequences(sinopsis_test)

In [13]:
# Melakukan padding pada sequence
padded_latih = pad_sequences(sekuens_latih)
padded_test = pad_sequences(sekuens_test)

In [14]:
import tensorflow as tf
# Membuat model menggunakan arsitektur Sequential
model = tf.keras.Sequential([
    # Layer embedding dengan input_dim=5000 dan output_dim=16
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    # Layer LSTM dengan 64 unit
    tf.keras.layers.LSTM(64),
    # Layer Dense dengan 128 unit dan fungsi aktivasi 'relu'
    tf.keras.layers.Dense(128, activation='relu'),
    # Layer Dense dengan 64 unit dan fungsi aktivasi 'relu'
    tf.keras.layers.Dense(64, activation='relu'),
    # Layer output Dense dengan 5 unit dan fungsi aktivasi 'softmax'
    tf.keras.layers.Dense(5, activation='softmax')
])

In [15]:
# Meng-compile model dengan loss function 'categorical_crossentropy', optimizer 'adam', dan metrik 'accuracy'
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [16]:
# Melatih model selama 30 epoch dan menggunakan validation data
num_epochs = 30
history = model.fit(padded_latih, label_latih, epochs=num_epochs, validation_data=(padded_test, label_test), verbose=2)

Epoch 1/30
26/26 - 21s - loss: 1.6098 - accuracy: 0.2027 - val_loss: 1.6096 - val_accuracy: 0.1791 - 21s/epoch - 795ms/step
Epoch 2/30
26/26 - 14s - loss: 1.6206 - accuracy: 0.2512 - val_loss: 1.6090 - val_accuracy: 0.2289 - 14s/epoch - 525ms/step
Epoch 3/30
26/26 - 14s - loss: 1.5938 - accuracy: 0.2127 - val_loss: 1.6073 - val_accuracy: 0.2189 - 14s/epoch - 520ms/step
Epoch 4/30
26/26 - 14s - loss: 1.5367 - accuracy: 0.3619 - val_loss: 1.5937 - val_accuracy: 0.2836 - 14s/epoch - 526ms/step
Epoch 5/30
26/26 - 12s - loss: 1.2691 - accuracy: 0.5137 - val_loss: 1.6449 - val_accuracy: 0.3035 - 12s/epoch - 476ms/step
Epoch 6/30
26/26 - 14s - loss: 0.7953 - accuracy: 0.6654 - val_loss: 1.8742 - val_accuracy: 0.2786 - 14s/epoch - 544ms/step
Epoch 7/30
26/26 - 15s - loss: 0.4918 - accuracy: 0.8383 - val_loss: 2.5292 - val_accuracy: 0.3234 - 15s/epoch - 569ms/step
Epoch 8/30
26/26 - 12s - loss: 0.2624 - accuracy: 0.9179 - val_loss: 2.6947 - val_accuracy: 0.3234 - 12s/epoch - 479ms/step
Epoch 9/